Processing Steps
---
1. Features Engineering
- parameters of Algorithms

In [1]:
import numpy as np
import pandas as pd
import matplotlib,os
from stockstats import StockDataFrame as Sdf
# load Lasso
from sklearn.linear_model import  Lasso ,MultiTaskElasticNet
# load NN functions
from keras.models import Sequential
from keras.layers.core import Dense, Activation

from xgboost import XGBRegressor,plot_importance


import matplotlib.pyplot as plt
%matplotlib inline

/Users/cch/anaconda36/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
df0050_date=pd.read_csv("yahoo/20180511/0050.csv",index_col='Date') 

In [15]:
df0050_date.tail(7)

,Open,High,Low,Close,Volume,weekday,code,yesterday_close,yesterday_volume,price_change,volume_Change,sign
Date,,,,,,,,,,,,
2018-05-10,80.70,80.70,80.45,80.65,5977.000000,3,50,80.45,3359.000000,0.20,2618.000000,1
2018-05-11,81.10,81.70,81.05,81.60,17654.000000,4,50,80.65,5977.000000,0.95,11677.000000,1
2018-05-14,85.43,85.74,85.04,85.43,3059.522631,0,50,81.60,17654.000000,3.83,-14594.477369,1
2018-05-15,85.48,85.79,85.09,85.48,2822.583315,1,50,85.43,3059.522631,0.05,-236.939317,1
2018-05-16,85.53,85.85,85.14,85.53,2803.552445,2,50,85.48,2822.583315,0.05,-19.030870,1
2018-05-17,85.58,85.90,85.20,85.58,2891.492010,3,50,85.53,2803.552445,0.05,87.939566,1
2018-05-18,85.63,85.95,85.25,85.63,2957.947264,4,50,85.58,2891.492010,0.05,66.455254,1


In [16]:
df0050_date_1=df0050_date.copy()
stock_df = Sdf.retype(df0050_date_1)
#df0050_date['rsi5']=stock_df['rsi_5']

Volatility
---

1. $\mathbf{\text{ret}_i= \log x_i -\log x_{i-1}}$
-  Volatility=$\mathbf{\sum_w \text{std}(\text{ret}_i)/w}$, where $w$ is the length of considered period

In [27]:
def generate_features(df):
    """ Generate features for a stock/index based on historical price and performance
    Args:
        df (dataframe with columns "Open", "Close", "High", "Low", "Volume", "Adjusted Close")
    Returns:
        dataframe, data set with new features
    """
    df_new = pd.DataFrame()
    # 6 original features
    df_new['High'] = df['High']
    df_new['Low'] = df['Low']
    df_new['weekday'] = df['weekday']
    df_new['code'] = df['code']
    #df_new['price_change'] = df['price_Change']
    df_new['sign'] = df['sign']
    
    
    df_new['Open'] = df['Open']
    df_new['open_1'] = df['Open'].shift(1)
    df_new['close_1'] = df['Close'].shift(1)
    df_new['high_1'] = df['High'].shift(1)
    df_new['low_1'] = df['Low'].shift(1)
    df_new['volume_1'] = df['Volume'].shift(1)
    # 31 original features
    # average price
    df_new['avg_price_5'] = pd.rolling_mean(df['Close'], window=5).shift(1)
    df_new['avg_price_30'] = pd.rolling_mean(df['Close'], window=21).shift(1)
    df_new['avg_price_365'] = pd.rolling_mean(df['Close'], window=252).shift(1)
    df_new['ratio_avg_price_5_30'] = df_new['avg_price_5'] / df_new['avg_price_30']
    df_new['ratio_avg_price_5_365'] = df_new['avg_price_5'] / df_new['avg_price_365']
    df_new['ratio_avg_price_30_365'] = df_new['avg_price_30'] / df_new['avg_price_365']
    # average volume
    df_new['avg_volume_5'] = pd.rolling_mean(df['Volume'], window=5).shift(1)
    df_new['avg_volume_30'] = pd.rolling_mean(df['Volume'], window=21).shift(1)
    df_new['avg_volume_365'] = pd.rolling_mean(df['Volume'], window=252).shift(1)
    df_new['ratio_avg_volume_5_30'] = df_new['avg_volume_5'] / df_new['avg_volume_30']
    df_new['ratio_avg_volume_5_365'] = df_new['avg_volume_5'] / df_new['avg_volume_365']
    df_new['ratio_avg_volume_30_365'] = df_new['avg_volume_30'] / df_new['avg_volume_365']
    # standard deviation of prices
    df_new['std_price_5'] = pd.rolling_std(df['Close'], window=5).shift(1)
    df_new['std_price_30'] = pd.rolling_std(df['Close'], window=21).shift(1)
    df_new['std_price_365'] = pd.rolling_std(df['Close'], window=252).shift(1)
    df_new['ratio_std_price_5_30'] = df_new['std_price_5'] / df_new['std_price_30']
    df_new['ratio_std_price_5_365'] = df_new['std_price_5'] / df_new['std_price_365']
    df_new['ratio_std_price_30_365'] = df_new['std_price_30'] / df_new['std_price_365']
    # standard deviation of volumes
    df_new['std_volume_5'] = pd.rolling_std(df['Volume'], window=5).shift(1)
    df_new['std_volume_30'] = pd.rolling_std(df['Volume'], window=21).shift(1)
    df_new['std_volume_365'] = pd.rolling_std(df['Volume'], window=252).shift(1)
    df_new['ratio_std_volume_5_30'] = df_new['std_volume_5'] / df_new['std_volume_30']
    df_new['ratio_std_volume_5_365'] = df_new['std_volume_5'] / df_new['std_volume_365']
    df_new['ratio_std_volume_30_365'] = df_new['std_volume_30'] / df_new['std_volume_365']
    # # return
    df_new['return_1'] = ((df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)).shift(1)
    df_new['return_5'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)).shift(1)
    df_new['return_30'] = ((df['Close'] - df['Close'].shift(21)) / df['Close'].shift(21)).shift(1)
    df_new['return_365'] = ((df['Close'] - df['Close'].shift(252)) / df['Close'].shift(252)).shift(1)
    df_new['moving_avg_5'] = pd.rolling_mean(df_new['return_1'], window=5)
    df_new['moving_avg_30'] = pd.rolling_mean(df_new['return_1'], window=21)
    df_new['moving_avg_365'] = pd.rolling_mean(df_new['return_1'], window=252)
    
    df_new['Log Ret'] = np.log(df['Close']/df['Close'].shift(1))
    w = 63 # ~ 1 year
    df_new['VolatilityQuater'] = df_new['Log Ret'].rolling(window=w, center=True).std()
    
    
    #RSI
    df_new['RSI6']=stock_df['rsi_6']
    df_new['VR']=stock_df['vr']
    df_new['macd']=stock_df['macd']
    
    # data from US stock market
    
    for US_index in US_indecies:
        featurename='%s' %USind[US_index]
        df_new[featurename]=df_yahoo[featurename]
    
    
    
    # the target
    df_new['Close'] = df['Close']
    df_new=df_new.fillna(method='ffill')
    df_new = df_new.dropna(axis=0)
    return df_new

In [28]:
data=generate_features(df0050_date)

In [19]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


Data Division

1. $X,y$: all features except *Close*, 'Close'
-  split into [train,val] set
- test for prediction

In [30]:
X_columns = list(data.drop(['Close'], axis=1).columns)
y_column = 'Close'

In [31]:
X = data[X_columns]
y = data[y_column]

In [32]:
val_ratio =0.2

X_train, X_val, y_train, y_val = train_test_split(X[:-5], y[:-5],
                                                  test_size=val_ratio,
                                                  random_state=42)
X_test=X[-5:]
Y_test=y[-5:]

Evaluation
---
1. $\mathbf{MSE}=\frac{1}{n}\sqrt{\sum{(y_i-y^{prec}_i)^2}}$: small (~0) for good, big for bad (~1)
- $\mathbf{MAE}=\frac{1}{n}{\sum{\left|y_i-y^{prec}_i\right|}}$: small (~0) for good, big for bad (~1)
- $\mathbf{R^2}$: small (~0) for bad, big for good (~1)
  - $\mathbf{SST=\text{Sum of tatal error}}=\sum(y_i-\bar y)^2$
  - $\mathbf{SSR=\text{Sum of Residues}}=\sum(y_i-y^{prec}_i)^2$
  - $\mathbf{R^2=1-\frac{SSR}{SST}}$, 

Parameters Chosen
---
1. define parameter set, 
- test 
- find out best 
- predict

In [33]:
from sklearn.model_selection import GridSearchCV
# First experiment with linear regression

# SGD is very sensitive to data with features at different scales. Hence we need to do feature scaling before training.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_val = scaler.transform(X_val)

param_grid = {
    "alpha": [1e-5, 3e-5, 1e-4],
    "eta0": [0.01, 0.03, 0.1],
}

from sklearn.linear_model import SGDRegressor
lr = SGDRegressor(penalty='l2', n_iter=1000)
grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_scaled_train, y_train)

print(grid_search.best_params_)

lr_best = grid_search.best_estimator_
# print(grid_search.best_score_)

predictions_lr = lr_best.predict(X_scaled_val)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_lr)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_lr)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_lr)))


{'alpha': 1e-05, 'eta0': 0.03}
MSE: 0.090
MAE: 0.220
R^2: 0.971


In [14]:
X_scaled_test = scaler.transform(X_test)
lr_best.predict(X_scaled_test)

array([82.18574336, 81.94858176, 83.36876597, 82.54054246, 81.76222658])

In [35]:
# Next experiment with random forest

param_grid = {
    "max_depth": [30, 50],
    "min_samples_split": [5, 10, 20],

}

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
# print(grid_search.best_score_)

rf_best = grid_search.best_estimator_
predictions_rf = rf_best.predict(X_val)

print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_rf)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_rf)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_rf)))


{'max_depth': 30, 'min_samples_split': 5}
MSE: 0.042
MAE: 0.180
R^2: 0.987


In [16]:
rf_best.predict(X_test)

array([82.36436744, 82.07018865, 83.11011908, 83.26431186, 82.3236982 ])

In [36]:
rf_best.predict(X_test)

array([82.26640095, 82.01824975, 83.08874581, 83.18531409, 82.30270682])

In [38]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

param_grid = {
    "learning_rate": [0.05,0.1,0.2],
    "max_depth": [5, 10, 15, 20],
    "colsample_bytree": [0.7,0.8,0.9],
}


model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
# print(grid_search.best_score_)

xgb_best = grid_search.best_estimator_
predictions_xgb = xgb_best.predict(X_val)

print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_xgb)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_xgb)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_xgb)))


{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5}
MSE: 0.078
MAE: 0.219
R^2: 0.975


In [39]:
 xgb_best.predict(X_test)

array([81.85501, 81.95592, 83.12248, 82.64492, 82.26562], dtype=float32)

In [18]:
 xgb_best.predict(X_test)

array([82.05426, 82.04069, 83.25623, 82.76489, 82.4588 ], dtype=float32)

In [19]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

param_grid = {
    "learning_rate": [0.05,0.1,0.2],
    "max_depth": [5, 10, 15, 20],
    "colsample_bytree": [0.7,0.8,0.9],
}

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_val = scaler.transform(X_val)


model_xgb = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=90, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =15)

grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_scaled_train, y_train)

print(grid_search.best_params_)
# print(grid_search.best_score_)

xgb_best = grid_search.best_estimator_
predictions_xgb = xgb_best.predict(X_scaled_val)

print('MSE: {0:.3f}'.format(mean_squared_error(y_val, predictions_xgb)))
print('MAE: {0:.3f}'.format(mean_absolute_error(y_val, predictions_xgb)))
print('R^2: {0:.3f}'.format(r2_score(y_val, predictions_xgb)))


{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 5}
MSE: 0.044
MAE: 0.152
R^2: 0.986


In [20]:
X_scaled_test = scaler.transform(X_test)
xgb_best.predict(X_scaled_test)

array([82.05426, 82.04069, 83.25623, 82.76489, 82.4588 ], dtype=float32)